# Creazione matrici per ogni sottopercorso in base alla mossa

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

path = "data/DB-Output_original.csv"
data = pd.read_csv(path)

data = data.loc[:, ['Initial Solution', 'Moves', 'OFIS', 'OFFS', 'OF_Diff']]
data = data[data.OF_Diff != 0]
#data = data.reset_index(drop=False)

print(data.shape)
data.head(10)

(9248, 5)


,Initial Solution,Moves,OFIS,OFFS,OF_Diff
5,"[['D0', 'C61', 'D0'], ['D0', 'C24', 'C50', 'D0...","['null', 'null', 'ProbabilisticWorstRemovalCus...",30978.64707,26301.09614,4677.550934
21,"[['D0', 'C24', 'D0'], ['D0', 'C26', 'C25', 'D0...","['null', 'null', 'WorstTimeDestroyCustomer', '...",26301.09614,23702.14047,2598.955662
30,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C6', 'C91...","['null', 'null', 'WorstTimeDestroyCustomer', '...",23702.14047,18726.22750,4975.912973
37,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C91', 'C6...","['null', 'null', 'RandomRouteDestroyCustomer',...",18726.22750,17507.99998,1218.227524
40,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C91', 'D0...","['null', 'null', 'TimeBasedDestroyCustomer', '...",17507.99998,17471.76518,36.234795
43,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C91', 'D0...","['null', 'null', 'WorstDistanceDestroyCustomer...",17471.76518,17469.41941,2.345772
46,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C6', 'C24...","['null', 'null', 'WorstDistanceDestroyCustomer...",17469.41941,16251.78228,1217.637127
47,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C24', 'C6...","['null', 'null', 'DemandBasedDestroyCustomer',...",16251.78228,16249.43924,2.343046
49,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C24', 'C6...","['RandomDestroyStation', 'DeterministicBestRep...",16249.43924,16221.64776,27.791482
50,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C24', 'C6...","['null', 'null', 'WorstTimeDestroyCustomer', '...",16221.64776,15005.40910,1216.238655


# Lista delle mosse disponibili e applicate

A partire dal dataframe iniziale, si crea una lista chiamata "mv", dove al suo interno sono presenti tutte le diverse mosse applicate e riportate nella tabella.

In [2]:
mv = data["Moves"]
moves = []
for row in mv:
    for elem in row.split("', '"):
        moves.append(elem)

for i in range(len(moves)):
    moves[i] = moves[i].replace("['","")
    moves[i] = moves[i].replace("']","")

to_remove = "null"
moves = list(filter(lambda x: x != to_remove, moves))

moves=list(set(moves))

print("Lista delle", len(moves), "possibili e diverse mosse: \n", moves)

Lista delle 16 possibili e diverse mosse: 
 ['RandomDestroyStation', 'DeterministicBestRepairStation', 'ShawDestroyCustomer', 'ProbabilisticBestRepairStation', 'RandomRouteDestroyCustomer', 'TimeBasedDestroyCustomer', 'WorstTimeDestroyCustomer', 'ProbabilisticWorstRemovalCustomer', 'DemandBasedDestroyCustomer', 'ProximityBasedDestroyCustomer', 'ZoneDestroyCustomer', 'WorstDistanceDestroyCustomer', 'GreedyRepairCustomer', 'GreedyRouteRemoval', 'LongestWaitingTimeDestroyStation', 'GreedyDestroyCustomer']


In [3]:
data["Moves"] = data["Moves"].str.replace("[","")
data["Moves"] = data["Moves"].str.replace("]","")
data["Moves"] = data["Moves"].str.replace("'","")
data.head(10)

,Initial Solution,Moves,OFIS,OFFS,OF_Diff
5,"[['D0', 'C61', 'D0'], ['D0', 'C24', 'C50', 'D0...","null, null, ProbabilisticWorstRemovalCustomer,...",30978.64707,26301.09614,4677.550934
21,"[['D0', 'C24', 'D0'], ['D0', 'C26', 'C25', 'D0...","null, null, WorstTimeDestroyCustomer, GreedyRe...",26301.09614,23702.14047,2598.955662
30,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C6', 'C91...","null, null, WorstTimeDestroyCustomer, GreedyRe...",23702.14047,18726.22750,4975.912973
37,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C91', 'C6...","null, null, RandomRouteDestroyCustomer, Greedy...",18726.22750,17507.99998,1218.227524
40,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C91', 'D0...","null, null, TimeBasedDestroyCustomer, GreedyRe...",17507.99998,17471.76518,36.234795
43,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C91', 'D0...","null, null, WorstDistanceDestroyCustomer, Gree...",17471.76518,17469.41941,2.345772
46,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C6', 'C24...","null, null, WorstDistanceDestroyCustomer, Gree...",17469.41941,16251.78228,1217.637127
47,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C24', 'C6...","null, null, DemandBasedDestroyCustomer, Greedy...",16251.78228,16249.43924,2.343046
49,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C24', 'C6...","RandomDestroyStation, DeterministicBestRepairS...",16249.43924,16221.64776,27.791482
50,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C24', 'C6...","null, null, WorstTimeDestroyCustomer, GreedyRe...",16221.64776,15005.40910,1216.238655


In [4]:
user_input = input("Inserisci una delle possibili mosse: ")
data = data.loc[data['Moves'].str.contains(user_input)]
data

,Initial Solution,Moves,OFIS,OFFS,OF_Diff
21,"[['D0', 'C24', 'D0'], ['D0', 'C26', 'C25', 'D0...","null, null, WorstTimeDestroyCustomer, GreedyRe...",26301.096140,23702.140470,2598.955662
30,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C6', 'C91...","null, null, WorstTimeDestroyCustomer, GreedyRe...",23702.140470,18726.227500,4975.912973
50,"[['D0', 'C26', 'C25', 'D0'], ['D0', 'C24', 'C6...","null, null, WorstTimeDestroyCustomer, GreedyRe...",16221.647760,15005.409100,1216.238655
51,"[['D0', 'C26', 'D0'], ['D0', 'C24', 'C25', 'C6...","null, null, WorstTimeDestroyCustomer, GreedyRe...",15005.409100,15004.666210,0.742891
64,"[['D0', 'C21', 'C25', 'C24', 'D0'], ['D0', 'C5...","RandomDestroyStation, ProbabilisticBestRepairS...",14991.375650,13775.899950,1215.475703
...,...,...,...,...,...
35233,"[['D0', 'C5', 'C10', 'C11', 'C14', 'C16', 'C19...","null, null, WorstTimeDestroyCustomer, GreedyRe...",37111.236150,36418.411040,692.825115
35236,"[['D0', 'C93', 'C21', 'D0'], ['D0', 'C10', 'C2...","null, null, WorstTimeDestroyCustomer, GreedyRe...",110907.095700,105568.926600,5338.169089
35337,"[['D0', 'C21', 'D0'], ['D0', 'C67', 'C66', 'D0...","null, null, WorstTimeDestroyCustomer, GreedyRe...",29341.683910,20865.501670,8476.182250
35395,"[['D0', 'C5', 'C2', 'C1', 'C14', 'C25', 'C21',...","LongestWaitingTimeDestroyStation, Probabilisti...",4811.496557,4808.175137,3.321419


Questi passaggi servono ad isolare i luoghi contenuti nella colonna "Initial Solution", in modo da poter costruire una matrice a seconda della mossa pressa in considerazione 

In [5]:
init_sol = data['Initial Solution']
init_sol.head(10)

#isolamento dei percorsi -> tutti i percorsi di tutti gli indici vengono inseriti nella lista sol
sol = []
for row in init_sol:
    for elem in row.split("[' "):
        sol.append(elem)

for i in range(len(sol)):
    sol[i] = sol[i].replace("[['","")
    sol[i] = sol[i].replace("']]","")
    sol[i] = sol[i].replace("'","")
    sol[i] = sol[i].replace(",","")
    sol[i] = sol[i].replace("[","")    
    
for i in range(len(sol)):
    sol[i] = sol[i].split('] ')

In [6]:
data["Initial Solution"] = data["Initial Solution"].str.replace("[","")
data["Initial Solution"] = data["Initial Solution"].str.replace("]","")
data["Initial Solution"] = data["Initial Solution"].str.replace(",","")
data["Initial Solution"] = data["Initial Solution"].str.replace("'","")

init_sol = data["Initial Solution"]

places = []
for row in init_sol:
    for elem in row.split():
        places.append(elem)

places = np.array(places)
distinct_places = np.unique(places)

print(distinct_places)
print(len(distinct_places))


['C1' 'C10' 'C100' 'C11' 'C12' 'C13' 'C14' 'C15' 'C16' 'C17' 'C18' 'C19'
 'C2' 'C20' 'C21' 'C22' 'C23' 'C24' 'C25' 'C26' 'C27' 'C28' 'C29' 'C3'
 'C30' 'C31' 'C32' 'C33' 'C34' 'C35' 'C36' 'C37' 'C38' 'C39' 'C4' 'C40'
 'C41' 'C42' 'C43' 'C44' 'C45' 'C46' 'C47' 'C48' 'C49' 'C5' 'C50' 'C51'
 'C52' 'C53' 'C54' 'C55' 'C56' 'C57' 'C58' 'C59' 'C6' 'C60' 'C61' 'C62'
 'C63' 'C64' 'C65' 'C66' 'C67' 'C68' 'C69' 'C7' 'C70' 'C71' 'C72' 'C73'
 'C74' 'C75' 'C76' 'C77' 'C78' 'C79' 'C8' 'C80' 'C81' 'C82' 'C83' 'C84'
 'C85' 'C86' 'C87' 'C88' 'C89' 'C9' 'C90' 'C91' 'C92' 'C93' 'C94' 'C95'
 'C96' 'C97' 'C98' 'C99' 'D0' 'S0' 'S1' 'S10' 'S11' 'S12' 'S13' 'S14'
 'S15' 'S16' 'S17' 'S18' 'S19' 'S2' 'S20' 'S3' 'S4' 'S5' 'S7' 'S9']
120


In [7]:
# lista vuota
lista_principale = [] # contiene 906 elementi, ogni elemento contiene n dataframe relativi all'n-esimo indice

for index_ext, value_ext in enumerate(sol):
    
    lista_dataframe = []
    for index_int, value_int in enumerate(value_ext):

        #print(type(value_ext, value_int))
        zeros = np.zeros([len(distinct_places), len(distinct_places)])
        df = pd.DataFrame(zeros, index=distinct_places, columns=distinct_places)

        temp_string = value_int

        split = temp_string.split(" ")
        split = np.array(split)

        for i in range(len(split)-1):

            df.loc[split[i], split[i+1]] = 1

        lista_dataframe.append(df)

    # somma dei dataframe risultanti in un unico dataframe
    df_sum = lista_dataframe[0]
    for i in range (1, len(lista_dataframe)):
        df_sum = df_sum.add(lista_dataframe[i])

    # inserimento del dataframe somma nella lista principale
    lista_principale.append(df_sum)

Conversione dei dataframe in matrici numpy.

In [8]:
for i in range(len(lista_principale)):
    lista_principale[i] = lista_principale[i].to_numpy()
    
print(type(lista_principale[0]), len(lista_principale))

<class 'numpy.ndarray'> 960


In [9]:
print(type(lista_principale[0]), len(lista_principale))
lista_principale[0].shape

<class 'numpy.ndarray'> 960


(120, 120)

Ricapitolando, a partire dalla scelta di una mossa (tra le 16 disponibili), si prendono tutte le iterazioni dell'algoritmo che applicano la mossa in questione.

Vengono dunque generate le matrici di adiacenza per ogni sottopercorso nel quale venga applicata la mossa scelta, e sommate tutte le matrici in un'unica risultante per iterazione.

Alla fine, in "lista_principale", si otterrà una lista contenente le matrici di adiacenza risultanti da tutte le iterazioni.

# Esempio iterazione del notebook

- si prende il dataframe principale

- si sceglie una delle possibili 16 mosse (es. 'ShawDestroyCustomer', 'ProximityBasedDestroyCustomer', 'GreedyRouteRemoval'...)

- il dataframe originale viene ridotto alle sole interazioni dove è stata applicata la mossa scelta

- per ogni riga del dataframe abbiamo un'iterazione che comprende diversi sottopercorsi

- si genera una (sotto)matrice di adiacenza per ogni sottopercorso (quindi si hanno n matrici per riga)

- ogni (sotto)matrice viene sommata in un'unica matrice risultante (quindi una per riga)

- le matrici di adiacenza risultanti e relative ad una specifica mossa vengono inserite in una lista.

In [12]:
from PIL import Image

for n in range (len(lista_principale)):
    
    img = Image.fromarray(lista_principale[n])
    img = img.convert("L")
    img.save(f"images/file_{n}.png")

In [13]:
length = len(lista_principale)
a = np.zeros(shape=(length))

for i in range(length):
    img = Image.open(f"images/file_{n}.png")
    a.append(img)
    

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [ ]:
(X_train, y_train), (X_test, y_test) = 

In [ ]:
import matplotlib.pyplot as plt
#plot the first image in the dataset
plt.imshow(X_train[0])

In [14]:
from tensorflow import keras
from sklearn.model_selection import train_test_split